# Create PAO1 and PA14 compendia

This notebook is using the observation from the [exploratory notebook](../0_explore_data/cluster_by_accessory_gene.ipynb) to bin samples into PAO1 or PA14 compendia.

A sample is considered PAO1 if the median gene expression of PA14 accessory genes is 0 and PAO1 accessory genes in > 0.
Similarlty, a sample is considered PA14 if the median gene expression of PA14 accessory genes is > 0 and PAO1 accessory genes in 0.

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
from textwrap import fill
import matplotlib.pyplot as plt
from scripts import paths, utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# User param
# same_threshold: if median accessory expression of PAO1 samples > same_threshold then this sample is binned as PAO1
# 25 threshold based on comparing expression of PAO1 SRA-labeled samples vs non-PAO1 samples
same_threshold = 25

# opp_threshold: if median accessory expression of PA14 samples < opp_threshold then this sample is binned as PAO1
# 25 threshold based on previous plot (eye-balling trying to avoid samples
# on the diagonal of explore_data/cluster_by_accessory_gene.ipynb plot)
opp_threshold = 25

## Load data

The expression data being used is described in the [paper](link TBD) with source code [here](https://github.com/hoganlab-dartmouth/pa-seq-compendia)

In [12]:
# Expression data files
pao1_expression_filename = paths.PAO1_GE
pa14_expression_filename = paths.PA14_GE

# File containing table to map sample id to strain name
sample_to_strain_filename = paths.SAMPLE_TO_STRAIN

In [13]:
# Load expression data
pao1_expression = pd.read_csv(pao1_expression_filename, sep="\t", index_col=0, header=0)
pa14_expression = pd.read_csv(pa14_expression_filename, sep="\t", index_col=0, header=0)

In [14]:
# Load metadata
# Set index to experiment id, which is what we will use to map to expression data
sample_to_strain_table_full = pd.read_csv(sample_to_strain_filename, index_col=2)

## Get core and accessory annotations

In [15]:
# Annotations are from BACTOME
# Gene ids from PAO1 are annotated with the homologous PA14 gene id and vice versa
pao1_annot_filename = paths.GENE_PAO1_ANNOT
pa14_annot_filename = paths.GENE_PA14_ANNOT

core_acc_dict = utils.get_my_core_acc_genes(
    pao1_annot_filename, pa14_annot_filename, pao1_expression, pa14_expression
)

Number of PAO1 core genes: 5366
Number of PA14 core genes: 5363
Number of PAO1 core genes in my dataset: 5361
Number of PA14 core genes in my dataset: 5361
Number of PAO1-specific genes: 202
Number of PA14-specific genes: 530


In [16]:
pao1_acc = core_acc_dict["acc_pao1"]
pa14_acc = core_acc_dict["acc_pa14"]

In [17]:
pao1_acc_df = pd.DataFrame(pao1_acc)
pa14_acc_df = pd.DataFrame(pa14_acc)

In [18]:
# Save to files (supplementary data tables)
pao1_acc_df.to_csv("pao1_acc_gene_ids.tsv", sep="\t")
pa14_acc_df.to_csv("pa14_acc_gene_ids.tsv", sep="\t")

## Format expression data

Format index to only include experiment id. This will be used to map to expression data and SRA labels later

In [8]:
# Format expression data indices so that values can be mapped to `sample_to_strain_table`
pao1_index_processed = pao1_expression.index.str.split(".").str[0]
pa14_index_processed = pa14_expression.index.str.split(".").str[0]

print(
    f"No. of samples processed using PAO1 reference after filtering: {pao1_expression.shape}"
)
print(
    f"No. of samples processed using PA14 reference after filtering: {pa14_expression.shape}"
)
pao1_expression.index = pao1_index_processed
pa14_expression.index = pa14_index_processed

No. of samples processed using PAO1 reference after filtering: (2333, 5563)
No. of samples processed using PA14 reference after filtering: (2333, 5891)


In [9]:
pao1_expression.head()

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA1905,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1
ERX541571,5621.037929,902.172829,1380.825635,1811.863764,140.337996,508.725234,483.664878,666.605479,681.641693,77.687105,...,0.0,97.735390,471.134699,12.530178,87.711247,75.181069,77.687105,2288.010535,250.603564,0.000000
ERX541572,6323.898054,835.453446,1754.599065,1814.798699,190.876886,418.460865,321.554138,493.343336,666.600816,45.516796,...,0.0,80.755606,487.470201,10.277986,70.477619,46.985080,60.199633,2218.576726,199.686588,0.000000
ERX541573,4954.119979,893.818410,1984.802645,1752.146970,350.955170,362.785119,373.300630,465.311348,616.471815,42.062043,...,0.0,114.356179,782.091108,19.716583,153.789344,43.376482,106.469546,1474.800376,101.211790,0.000000
ERX541574,4603.356163,773.573295,2227.891090,1911.321096,311.809544,323.710671,435.581271,435.581271,506.988037,78.547442,...,0.0,152.334434,561.733224,21.422030,85.688119,38.083608,64.266089,2116.020491,78.547442,2.380226
ERX541575,4260.451254,875.408119,2140.106080,1788.361959,320.126671,288.509222,276.652679,577.018444,739.057871,57.306626,...,0.0,136.350249,677.799063,15.808725,195.632966,21.736996,118.565434,1650.035620,61.258807,0.000000


In [10]:
pa14_expression.head()

,PA14_55610,PA14_55600,PA14_55590,PA14_55580,PA14_55570,PA14_55560,PA14_55550,PA14_55540,PA14_55530,PA14_55520,...,PA14_19205,PA14_17675,PA14_67975,PA14_36345,PA14_43405,PA14_38825,PA14_24245,PA14_28895,PA14_55117,PA14_59845
ERX541571,209.353167,55.650842,0.000000,2.650040,10.600160,10.600160,18.550281,71.551082,66.251002,5.300080,...,42.400641,140.452125,3487.452760,180.202726,21.200321,2.650040,1014.965355,386.905853,151.052285,0.0
ERX541572,223.627085,52.077540,6.126769,15.316924,21.443693,9.190154,12.253539,79.648003,82.711388,3.063385,...,61.267694,180.739699,2977.609951,223.627085,27.570463,15.316924,1203.910196,572.852943,119.472004,0.0
ERX541573,167.343061,44.624816,18.128832,23.706934,8.367153,18.128832,5.578102,64.148173,57.175546,8.367153,...,55.781020,207.784301,1811.488635,68.331750,4.183577,12.550730,1540.950687,616.380275,62.753648,0.0
ERX541574,203.038931,13.454387,6.115630,14.677513,4.892504,17.123765,8.561883,47.701918,58.710052,7.338757,...,67.271935,226.278327,2239.543876,77.056944,6.115630,12.231261,1700.145269,895.328300,165.122023,0.0
ERX541575,193.971582,46.384509,8.433547,29.517415,8.433547,12.650321,4.216774,44.276122,42.167735,6.325160,...,40.059348,250.898025,1574.964910,65.359990,4.216774,10.541934,1313.524952,710.526338,145.478686,0.0


In [11]:
# Save pre-binned expression data
pao1_expression.to_csv(paths.PAO1_PREBIN_COMPENDIUM, sep="\t")
pa14_expression.to_csv(paths.PA14_PREBIN_COMPENDIUM, sep="\t")

## Bin samples as PAO1 or PA14

In [12]:
# Create accessory df
# accessory gene ids | median accessory expression | strain label

# PAO1
pao1_acc_expression = pao1_expression[pao1_acc]
pao1_acc_expression["median_acc_expression"] = pao1_acc_expression.median(axis=1)

# PA14
pa14_acc_expression = pa14_expression[pa14_acc]
pa14_acc_expression["median_acc_expression"] = pa14_acc_expression.median(axis=1)

pao1_acc_expression.head()

/home/alexandra/anaconda3/envs/core_acc/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/alexandra/anaconda3/envs/core_acc/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,PA0445,PA1386,PA2733,PA2234,PA4638,PA1150,PA2219,PA3159,PA0826.1,PA0135,...,PA2460,PA2225,PA3218,PA2731,PA1223,PA2230,PA3497,PA3974,PA2188,median_acc_expression
ERX541571,1393.355813,1137.740178,305.736347,112.771604,360.869131,1305.644566,40.096570,5149.903230,115372.868565,5.012071,...,170.410423,15.036214,0.000000,699.183942,37.590535,177.928530,117.783675,551.327840,20.048285,90.217283
ERX541572,1255.382595,635.766858,402.309744,114.526131,367.070934,1243.636325,30.833958,4052.463113,126244.503967,2.936567,...,192.345170,22.024256,2.936567,340.641827,73.414187,129.208969,89.565308,515.367592,14.682837,88.097024
ERX541573,1054.179948,311.522004,243.171185,39.433165,203.738020,2117.560967,48.634237,4359.993624,233310.264787,10.515511,...,177.449243,6.572194,2.628878,135.387200,159.047099,84.124086,65.721942,843.869734,15.773266,91.353499
ERX541574,1328.165842,407.018565,316.569995,64.266089,295.147965,3168.080172,73.786991,4512.907593,148933.091244,14.281353,...,221.360974,7.140677,14.281353,64.266089,211.840072,135.672855,90.448570,1094.903741,26.182481,103.539810
ERX541575,1199.486972,247.011320,241.083049,41.497902,219.346052,2165.795257,39.521811,4479.797306,168544.716287,3.952181,...,217.369962,3.952181,11.856543,154.135064,90.900166,69.163170,90.900166,792.412316,13.832634,82.995804


In [13]:
# Merge PAO1 and PA14 accessory dataframes
pao1_pa14_acc_expression = pao1_acc_expression.merge(
    pa14_acc_expression,
    left_index=True,
    right_index=True,
    suffixes=["_pao1", "_pa14"],
)

pao1_pa14_acc_expression.head()

,PA0445,PA1386,PA2733,PA2234,PA4638,PA1150,PA2219,PA3159,PA0826.1,PA0135,...,PA14_59380,PA14_22540,PA14_18070,PA14_15435,PA14_59350,PA14_03330,PA14_59670,PA14_22270,PA14_30990,median_acc_expression_pa14
ERX541571,1393.355813,1137.740178,305.736347,112.771604,360.869131,1305.644566,40.096570,5149.903230,115372.868565,5.012071,...,0.0,5.300080,37.100561,2.650040,2.650040,0.000000,0.000000,0.000000,0.000000,0.000000
ERX541572,1255.382595,635.766858,402.309744,114.526131,367.070934,1243.636325,30.833958,4052.463113,126244.503967,2.936567,...,0.0,0.000000,36.760617,3.063385,0.000000,0.000000,0.000000,0.000000,0.000000,3.063385
ERX541573,1054.179948,311.522004,243.171185,39.433165,203.738020,2117.560967,48.634237,4359.993624,233310.264787,10.515511,...,0.0,0.000000,50.202918,2.789051,2.789051,1.394526,2.789051,1.394526,1.394526,1.394526
ERX541574,1328.165842,407.018565,316.569995,64.266089,295.147965,3168.080172,73.786991,4512.907593,148933.091244,14.281353,...,0.0,2.446252,33.024405,2.446252,3.669378,1.223126,0.000000,1.223126,1.223126,1.223126
ERX541575,1199.486972,247.011320,241.083049,41.497902,219.346052,2165.795257,39.521811,4479.797306,168544.716287,3.952181,...,0.0,4.216774,46.384509,2.108387,0.000000,0.000000,2.108387,2.108387,0.000000,2.108387


In [14]:
# Find PAO1 samples
pao1_binned_ids = list(
    pao1_pa14_acc_expression.query(
        "median_acc_expression_pao1>@same_threshold & median_acc_expression_pa14<@opp_threshold"
    ).index
)

In [15]:
# Find PA14 samples
pa14_binned_ids = list(
    pao1_pa14_acc_expression.query(
        "median_acc_expression_pao1<@opp_threshold & median_acc_expression_pa14>@same_threshold"
    ).index
)

In [16]:
# Check that there are no samples that are binned as both PAO1 and PA14
shared_pao1_pa14_binned_ids = list(set(pao1_binned_ids).intersection(pa14_binned_ids))

if same_threshold == 25:
    assert len(shared_pao1_pa14_binned_ids) == 0

## Format SRA annotations

In [17]:
# Since experiments have multiple runs there are duplicated experiment ids in the index
# We will need to remove these so that the count calculations are accurate
sample_to_strain_table_full_processed = sample_to_strain_table_full[
    ~sample_to_strain_table_full.index.duplicated(keep="first")
]

assert (
    len(sample_to_strain_table_full.index.unique())
    == sample_to_strain_table_full_processed.shape[0]
)

In [18]:
# Aggregate boolean labels into a single strain label
aggregated_label = []
for exp_id in list(sample_to_strain_table_full_processed.index):
    if sample_to_strain_table_full_processed.loc[exp_id, "PAO1"].all() == True:
        aggregated_label.append("PAO1")
    elif sample_to_strain_table_full_processed.loc[exp_id, "PA14"].all() == True:
        aggregated_label.append("PA14")
    elif sample_to_strain_table_full_processed.loc[exp_id, "PAK"].all() == True:
        aggregated_label.append("PAK")
    elif (
        sample_to_strain_table_full_processed.loc[exp_id, "ClinicalIsolate"].all()
        == True
    ):
        aggregated_label.append("Clinical Isolate")
    else:
        aggregated_label.append("NA")

sample_to_strain_table_full_processed["Strain type"] = aggregated_label

sample_to_strain_table = sample_to_strain_table_full_processed["Strain type"].to_frame()

sample_to_strain_table.head()

/home/alexandra/anaconda3/envs/core_acc/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Strain type
Experiment,
SRX5057740,NA
SRX5057739,NA
SRX5057910,NA
SRX5057909,NA
SRX3573046,PAO1


## Save pre-binned data with median accessory expression
This dataset will be used for Georgia's manuscript, which describes how we generated these compendia

In [19]:
# Select columns with median accessory expression
pao1_pa14_acc_expression_select = pao1_pa14_acc_expression[
    ["median_acc_expression_pao1", "median_acc_expression_pa14"]
]

pao1_pa14_acc_expression_select.head()

,median_acc_expression_pao1,median_acc_expression_pa14
ERX541571,90.217283,0.000000
ERX541572,88.097024,3.063385
ERX541573,91.353499,1.394526
ERX541574,103.539810,1.223126
ERX541575,82.995804,2.108387


In [20]:
# Add SRA strain type
pao1_pa14_acc_expression_label = pao1_pa14_acc_expression_select.merge(
    sample_to_strain_table, left_index=True, right_index=True
)
# Rename column
pao1_pa14_acc_expression_label = pao1_pa14_acc_expression_label.rename(
    {"Strain type": "SRA label"}, axis=1
)

pao1_pa14_acc_expression_label.head()

,median_acc_expression_pao1,median_acc_expression_pa14,SRA label
ERX541571,90.217283,0.000000,NA
ERX541572,88.097024,3.063385,NA
ERX541573,91.353499,1.394526,NA
ERX541574,103.539810,1.223126,NA
ERX541575,82.995804,2.108387,NA


In [21]:
# Add our binned label
pao1_pa14_acc_expression_label["Our label"] = "NA"
pao1_pa14_acc_expression_label.loc[pao1_binned_ids, "Our label"] = "PAO1-like"
pao1_pa14_acc_expression_label.loc[pa14_binned_ids, "Our label"] = "PA14-like"

pao1_pa14_acc_expression_label.head()

,median_acc_expression_pao1,median_acc_expression_pa14,SRA label,Our label
ERX541571,90.217283,0.000000,NA,PAO1-like
ERX541572,88.097024,3.063385,NA,PAO1-like
ERX541573,91.353499,1.394526,NA,PAO1-like
ERX541574,103.539810,1.223126,NA,PAO1-like
ERX541575,82.995804,2.108387,NA,PAO1-like


In [22]:
# Confirm dimensions
pao1_expression_prebin_filename = paths.PAO1_PREBIN_COMPENDIUM
pa14_expression_prebin_filename = paths.PA14_PREBIN_COMPENDIUM

pao1_expression_prebin = pd.read_csv(
    pao1_expression_prebin_filename, sep="\t", index_col=0, header=0
)
pa14_expression_prebin = pd.read_csv(
    pa14_expression_prebin_filename, sep="\t", index_col=0, header=0
)

In [23]:
# he two expression prebins are because the same samples were mapped to 2 different references (PAO1 and a PA14 reference.
# This assertion is to make sure that the number of samples is the same in both, which it is.
# This assertion is also testing that when we added information about our accessory gene expression
# and labels we retained the same number of samples, which we did.
assert (
    pao1_expression_prebin.shape[0]
    == pa14_expression_prebin.shape[0]
    == pao1_pa14_acc_expression_label.shape[0]
)

In [24]:
# Save
pao1_pa14_acc_expression_label.to_csv(
    "prebinned_compendia_acc_expression.tsv", sep="\t"
)

## Create compendia

Create PAO1 and PA14 compendia

In [25]:
# Get expression data
# Note: reindexing needed here instead of .loc since samples from expression data
# were filtered out for low counts, but these samples still exist in log files
pao1_expression_binned = pao1_expression.loc[pao1_binned_ids]
pa14_expression_binned = pa14_expression.loc[pa14_binned_ids]

In [26]:
assert len(pao1_binned_ids) == pao1_expression_binned.shape[0]
assert len(pa14_binned_ids) == pa14_expression_binned.shape[0]

In [27]:
# Label samples with SRA annotations
# pao1_expression_label = pao1_expression_binned.join(
#    sample_to_strain_table, how='left')
pao1_expression_label = pao1_expression_binned.merge(
    sample_to_strain_table, left_index=True, right_index=True
)
pa14_expression_label = pa14_expression_binned.merge(
    sample_to_strain_table, left_index=True, right_index=True
)
print(pao1_expression_label.shape)
pao1_expression_label.head()

(890, 5564)


,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1,Strain type
ERX541571,5621.037929,902.172829,1380.825635,1811.863764,140.337996,508.725234,483.664878,666.605479,681.641693,77.687105,...,97.735390,471.134699,12.530178,87.711247,75.181069,77.687105,2288.010535,250.603564,0.000000,NA
ERX541572,6323.898054,835.453446,1754.599065,1814.798699,190.876886,418.460865,321.554138,493.343336,666.600816,45.516796,...,80.755606,487.470201,10.277986,70.477619,46.985080,60.199633,2218.576726,199.686588,0.000000,NA
ERX541573,4954.119979,893.818410,1984.802645,1752.146970,350.955170,362.785119,373.300630,465.311348,616.471815,42.062043,...,114.356179,782.091108,19.716583,153.789344,43.376482,106.469546,1474.800376,101.211790,0.000000,NA
ERX541574,4603.356163,773.573295,2227.891090,1911.321096,311.809544,323.710671,435.581271,435.581271,506.988037,78.547442,...,152.334434,561.733224,21.422030,85.688119,38.083608,64.266089,2116.020491,78.547442,2.380226,NA
ERX541575,4260.451254,875.408119,2140.106080,1788.361959,320.126671,288.509222,276.652679,577.018444,739.057871,57.306626,...,136.350249,677.799063,15.808725,195.632966,21.736996,118.565434,1650.035620,61.258807,0.000000,NA


In [28]:
print(pa14_expression_label.shape)
pa14_expression_label.head()

(505, 5892)


,PA14_55610,PA14_55600,PA14_55590,PA14_55580,PA14_55570,PA14_55560,PA14_55550,PA14_55540,PA14_55530,PA14_55520,...,PA14_17675,PA14_67975,PA14_36345,PA14_43405,PA14_38825,PA14_24245,PA14_28895,PA14_55117,PA14_59845,Strain type
ERX1477379,248.707957,13.753435,2.292239,264.753632,68.767177,22.922392,19.484033,55.013742,24.068512,2.292239,...,174.210182,1118.612747,264.753632,2.292239,11.461196,449.278890,293.406622,171.917943,181.086900,PA14
ERX2174773,56.232678,27.502445,8.594514,7.612284,22.836852,12.032320,5.402266,10.558975,56.723793,6.875611,...,46.655934,1115.076822,529.913187,0.000000,43.463686,135.056651,560.853438,73.421707,139.476687,PA14
ERX2174774,67.495419,21.860864,9.017607,6.284999,19.401517,12.023475,3.279130,16.395648,60.390638,4.372173,...,35.250644,1028.553671,532.038788,0.000000,49.733467,142.915401,738.897218,69.954766,160.404093,PA14
ERX2174775,58.088081,24.116927,7.001688,3.889827,18.411847,9.335584,3.889827,16.596595,46.159279,2.852540,...,54.716898,1157.612474,485.450392,0.518644,35.267764,168.040520,817.382285,56.532150,67.164344,PA14
ERX2174776,50.207404,23.036339,9.746143,4.430065,21.264312,10.632156,4.134727,12.994858,54.342132,2.362701,...,49.321391,1087.433313,650.924232,0.295338,50.798080,132.015940,611.348984,64.678950,124.041823,PA14


In [29]:
assert pao1_expression_binned.shape[0] == pao1_expression_label.shape[0]
assert pa14_expression_binned.shape[0] == pa14_expression_label.shape[0]

In [30]:
sample_to_strain_table["Strain type"].value_counts()

PAO1                861
NA                  795
Clinical Isolate    601
PA14                545
PAK                  65
Name: Strain type, dtype: int64

Looks like our binned compendium sizes is fairly close in number to what SRA annotates

## Quick comparison

Quick check comparing our binned labels compared with SRA annotations

In [31]:
pao1_expression_label["Strain type"].value_counts()

PAO1                639
NA                  202
Clinical Isolate     42
PA14                  7
Name: Strain type, dtype: int64

**Manually check that these PA14 are mislabeled**
* Clinical ones can be removed by increasing threshold

In [32]:
pa14_expression_label["Strain type"].value_counts()

PA14                434
NA                   46
Clinical Isolate     23
PAO1                  2
Name: Strain type, dtype: int64

## Check

Manually look up the samples we binned as PAO1 but SRA labeled as PA14. Are these cases of samples being mislabeled?

In [33]:
pao1_expression_label[pao1_expression_label["Strain type"] == "PA14"]

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA0195,PA4812,PA0195.1,PA0457.1,PA1552.1,PA1555.1,PA3701,PA4724.1,PA5471.1,Strain type
SRX4326016,1542.681036,2931.702520,710.485250,7267.640342,206.907910,141.488497,97.368428,1232.319170,731.784594,100.411192,...,509.662867,4440.913158,252.549361,56.291123,295.148048,803.289533,716.570777,54.769741,0.000000,PA14
SRX5099522,2448.870829,1784.319722,3556.179111,6235.150761,315.661776,348.889331,224.285999,835.673017,443.587864,66.455111,...,124.603333,3960.724597,25.751355,83.899577,546.593285,521.672619,1609.875057,22.428600,0.000000,PA14
SRX5099523,2517.289506,1854.938863,3429.583789,7284.071763,369.559523,342.779848,255.299574,1146.170116,480.248849,73.197780,...,71.412468,4515.053306,16.067805,83.909650,1031.910166,333.853289,1608.565848,7.141247,0.000000,PA14
SRX5099524,2541.397963,1589.397824,3784.241519,6160.964760,312.963901,385.879575,276.096426,797.156744,434.216931,87.662663,...,154.024119,3854.699361,22.120485,109.783149,565.301288,462.072357,1635.277348,23.759040,0.000000,PA14
SRX5290921,1682.100070,1315.199048,762.831575,2540.083998,225.785244,117.730877,703.966137,995.067827,263.684910,26.610404,...,147.566785,3011.813883,38.706042,62.897318,224.172493,1099.090314,810.407752,20.965773,14.514766,PA14
SRX7423386,1672.517997,2942.685374,1054.400381,6159.849851,153.169528,132.965656,642.293190,1203.511866,632.968326,81.678903,...,329.392192,4074.447591,52.236508,47.142369,343.552171,319.376598,714.301863,29.787761,0.000000,PA14
SRX7423388,1898.810933,3540.937123,1106.259685,6622.091411,151.386193,135.607034,366.826400,1165.158132,696.938915,101.236588,...,323.550884,2192.522020,50.305835,38.432408,383.542936,432.598936,720.841998,37.338803,0.000000,PA14


Note: These are the 7 PA14 labeled samples using threshold of 0

Most samples appear to be mislabeled:
* SRX5099522: https://www.ncbi.nlm.nih.gov/sra/?term=SRX5099522
* SRX5099523: https://www.ncbi.nlm.nih.gov/sra/?term=SRX5099523
* SRX5099524: https://www.ncbi.nlm.nih.gov/sra/?term=SRX5099524
* SRX5290921: https://www.ncbi.nlm.nih.gov/sra/?term=SRX5290921
* SRX5290922: https://www.ncbi.nlm.nih.gov/sra/?term=SRX5290922

Two samples appear to be PA14 samples treated with antimicrobial manuka honey.
* SRX7423386: https://www.ncbi.nlm.nih.gov/sra/?term=SRX7423386
* SRX7423388: https://www.ncbi.nlm.nih.gov/sra/?term=SRX7423388

In [34]:
pa14_label_pao1_binned_ids = list(
    pao1_expression_label[pao1_expression_label["Strain type"] == "PA14"].index
)
pao1_pa14_acc_expression.loc[
    pa14_label_pao1_binned_ids,
    ["median_acc_expression_pao1", "median_acc_expression_pa14"],
]

,median_acc_expression_pao1,median_acc_expression_pa14
SRX4326016,60.094577,6.158171
SRX5099522,111.727655,0.000000
SRX5099523,107.118702,0.000000
SRX5099524,107.734956,0.000000
SRX5290921,60.478190,0.000000
SRX7423386,84.139631,0.000000
SRX7423388,97.877658,0.000000


In [35]:
# Save compendia with SRA label
pao1_expression_label.to_csv(paths.PAO1_COMPENDIUM_LABEL, sep="\t")
pa14_expression_label.to_csv(paths.PA14_COMPENDIUM_LABEL, sep="\t")

# Save compendia without SRA label
pao1_expression_binned.to_csv(paths.PAO1_COMPENDIUM, sep="\t")
pa14_expression_binned.to_csv(paths.PA14_COMPENDIUM, sep="\t")

# Save processed metadata table
sample_to_strain_table.to_csv(paths.SAMPLE_TO_STRAIN_PROCESSED, sep="\t")